<a href="https://colab.research.google.com/github/jensullrich/DataScienceSS20/blob/Project/Project/Project_NLP_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Exploring Data

In [52]:
!pip install tensorflow
!pip install --upgrade tensorflow-hub
!pip install bert-for-tf2
!pip install sentencepiece

Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.6/dist-packages (0.8.0)


In [53]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone -b Project https://github.com/jensullrich/DataScienceSS20/
  path='DataScienceSS20/Project'

running in Colab: True
fatal: destination path 'DataScienceSS20' already exists and is not an empty directory.


In [54]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 9.7 GB  | Proc size: 5.8 GB
GPU RAM Free: 651MB | Used: 10790MB | Util  94% | Total 11441MB


In [55]:
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 9.7 GB  | Proc size: 5.8 GB
GPU RAM Free: 651MB | Used: 10790MB | Util  94% | Total 11441MB


In [80]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import bert
from sklearn.preprocessing import LabelEncoder

In [57]:
train=pd.read_csv(path + '/DATA/train.csv', index_col='id')
test=pd.read_csv(path + '/DATA/test.csv', index_col='id') # final test dataset for the kaggle competition
submission = pd.read_csv(path + '/DATA/sample_submission.csv')

In [58]:
train

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


## NaN Values

In [59]:
training_has_keyword = train["keyword"].notna().value_counts(normalize=True)
training_has_location = train["location"].notna().value_counts(normalize=True)
try:
    print(f'{training_has_keyword[True]*100:.1f}', "% of training data have keyword,", f'{(training_has_keyword[False])*100:.1f}', "% of values are missing")
    print(f'{training_has_keyword[True]*100:.1f}', "% of training data have location,", f'{(training_has_keyword[False])*100:.1f}', "% of values are missing")

    test_has_keyword = test["keyword"].notna().value_counts(normalize=True)
    test_has_location = test["location"].notna().value_counts(normalize=True)
    print(f'{test_has_keyword[True]*100:.1f}', "% of test data have keyword,", f'{(test_has_keyword[False])*100:.1f}', "% of values are missing")
    print(f'{test_has_location[True]*100:.1f}', "% of test data have location,", f'{(test_has_location[False])*100:.1f}', "% of values are missing")
except KeyError:
    print("NaN values already replaced")

99.2 % of training data have keyword, 0.8 % of values are missing
99.2 % of training data have location, 0.8 % of values are missing
99.2 % of test data have keyword, 0.8 % of values are missing
66.1 % of test data have location, 33.9 % of values are missing


In [60]:
print("Replacing NaN with missing_keyword and missing_location")
train["keyword"] = train["keyword"].fillna('missing_keyword')
train["location"] = train["location"].fillna('missing_location')
test["keyword"] = test["keyword"].fillna('missing_keyword')
test["location"] = test["location"].fillna('missing_location')

Replacing NaN with missing_keyword and missing_location


## Duplicate Values

In [61]:
duplicates = train.groupby(['text']).count().sort_values(by='target', ascending=False)
duplicates = duplicates[duplicates['target']>1]
duplicates.shape

(69, 3)

## Exploration

In [62]:
print("We have", len(train["keyword"]), "tweets with", train["keyword"].nunique(), "unique keywords.")
print("We have", len(train["location"]), "tweets with", train["location"].nunique(), "unique locations.")

We have 7613 tweets with 222 unique keywords.
We have 7613 tweets with 3342 unique locations.


In [63]:
grouped = train.groupby(train["keyword"])
grouped.size().sort_values()

grouped = train.groupby(train["location"])
grouped.target.sum().sort_values(ascending=False).head(10)

location
missing_location    1075
USA                   67
United States         27
Nigeria               22
India                 20
Mumbai                19
UK                    16
New York              16
London                16
Washington, DC        15
Name: target, dtype: int64

In [64]:
labelencoder = LabelEncoder()
train['encoded_keyword'] = labelencoder.fit_transform(train['keyword'])
train[20:60]

,keyword,location,text,target,encoded_keyword
id,,,,,
31,missing_keyword,missing_location,this is ridiculous....,0,145
32,missing_keyword,missing_location,London is cool ;),0,145
33,missing_keyword,missing_location,Love skiing,0,145
34,missing_keyword,missing_location,What a wonderful day!,0,145
36,missing_keyword,missing_location,LOOOOOOL,0,145
37,missing_keyword,missing_location,No way...I can't eat that shit,0,145
38,missing_keyword,missing_location,Was in NYC last week!,0,145
39,missing_keyword,missing_location,Love my girlfriend,0,145
40,missing_keyword,missing_location,Cooool :),0,145


# Training

In [65]:
training_parameters = {
    "learning_rate": 0.0001,
    "epochs": 10,
    "batch_size": 32
}

In [66]:
def bert_encode(tweet_texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in tweet_texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [82]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    meta_input = Input(shape=(222,), dtype=tf.int32, name='meta_input')

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = concatenate([sequence_output[:, 0, :], meta_input])
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [68]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 6.5 s, sys: 722 ms, total: 7.22 s
Wall time: 7.21 s


In [69]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
FullTokenizer=bert.bert_tokenization.FullTokenizer
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [70]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [83]:
model = build_model(bert_layer, max_len=160)
model.summary()

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

TypeError: ignored

In [ ]:
train_history = model.fit(
    train_input, train.encoded_keyword, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=12
)

In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)

In [ ]:
printm()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My Drive/INFM/DataScience/Project'

In [ ]:
model.save(root_path + '/models/model_1.h5') 